# Animal Vision - Model Script

# 1. Create Dataset and Dataloaders - data_setup.py

In [ ]:
%%writefile data_setup.py
"""
Contains functionality for creating 
PyTorch datasets and dataloaders for
Image classification data
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int,
                       num_workers: int = NUM_WORKERS):
    """
    Create training and testing dataloaders.

    Args:
      train_dir: Train directory path,
      test_dir: Test directory path,
      transform: torchvision transforms compose class to transform the data,
      batch_size: Total sample per batch,
      num_workers: An integer for the number of workers durning dataloaders

    Return:
      Returns a tuple of (train_dataloaders, test_dataloaders, class_names).
    """
    # Create datasets using ImageFolder
    train_data = datasets.ImageFolder(root=train_dir, 
                                      transform=transform)
    test_data = datasets.ImageFolder(root=test_dir, 
                                     transform=transform)
    
    # Get class name
    class_names = train_data.classes

    # Create dataloaders
    train_dataloaders = DataLoader(dataset=train_data,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   num_workers=num_workers,
                                   pin_memory=True)
    test_dataloaders = DataLoader(dataset=test_data,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  num_workers=num_workers,
                                  pin_memory=True)

    return train_dataloaders, test_dataloaders, class_names

Writing data_setup.py


In [ ]:
import data_setup
import os
from torchvision import transforms
train_dir = 'data/lion_tiger_wolf/train'
test_dir = 'data/lion_tiger_wolf/val'
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
BATCH_SIZE = 4
NUM_WORKERS = os.cpu_count()
train_dataloaders, test_dataloaders, class_names =  data_setup.create_dataloaders(train_dir,
                                                                                  test_dir,
                                                                                  transform,
                                                                                  BATCH_SIZE,
                                                                                  NUM_WORKERS)
train_dataloaders, test_dataloaders, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7fd895789b80>,
 ['LION', 'TIGER', 'WOLF'])

# 2. Creating a Model script - model_builder.py

In [ ]:
%%writefile model_builder.py
"""
Contains PyTorch Model code to 
initiate a custom model to train and test a dataset 
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates a neural network architecture similar to the TinyVGG architecture from
    here: https://github.com/poloclub/cnn-explainer/tree/master/tiny-vgg and for visuals 
    here: https://poloclub.github.io/cnn-explainer/

    Args:
      input_shape: An integer for the input units. E.g. 3 for RBG and 1 for grayscale.
      output_shape: An integer for the output units. E.g. Any number that represents your total labels.
      hidden_units: An integer for the hidden units. E.g. Any number to change the filters in the layer.

    Return:
      A tensor of logits with the length provided as per the output_shape.
    """
    def __init__(self, input_shape: int, output_shape: int, hidden_units: int):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*13*13,
                      output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x)
    return x

Writing model_builder.py


In [ ]:
import model_builder
import torch
from torchsummary import summary
device='cuda' if torch.cuda.is_available() else 'cpu'
baseline_model = model_builder.TinyVGG(input_shape=3,
                                       output_shape=len(class_names),
                                       hidden_units=20).to(device)
summary(baseline_model,(3, 64, 64), 4, device=device), baseline_model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [4, 20, 62, 62]             560
              ReLU-2            [4, 20, 62, 62]               0
            Conv2d-3            [4, 20, 60, 60]           3,620
              ReLU-4            [4, 20, 60, 60]               0
         MaxPool2d-5            [4, 20, 30, 30]               0
            Conv2d-6            [4, 20, 28, 28]           3,620
              ReLU-7            [4, 20, 28, 28]               0
            Conv2d-8            [4, 20, 26, 26]           3,620
              ReLU-9            [4, 20, 26, 26]               0
        MaxPool2d-10            [4, 20, 13, 13]               0
          Flatten-11                  [4, 3380]               0
           Linear-12                     [4, 3]          10,143
Total params: 21,563
Trainable params: 21,563
Non-trainable params: 0
---------------------------------

(None, TinyVGG(
   (conv_block1): Sequential(
     (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1))
     (1): ReLU()
     (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
     (3): ReLU()
     (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (conv_block2): Sequential(
     (0): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
     (1): ReLU()
     (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
     (3): ReLU()
     (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (classifier): Sequential(
     (0): Flatten(start_dim=1, end_dim=-1)
     (1): Linear(in_features=3380, out_features=3, bias=True)
   )
 ))

# 3. Create the training and testing script - engine.py

In [ ]:
%%writefile engine.py
"""
Contains training and testing function in PyTorch
"""
import torch
from torch.utils.tensorboard.writer import SummaryWriter
from datetime import datetime
import os
from tqdm.auto import tqdm
from typing import List, Dict, Tuple

# Function for training the model
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """
  Trains a PyTorch model.
  Turns the model to a training mode and applies steps
  like loss, accuracy metrics, optimizer.
  Args:
    model: A PyTorch model to be trained.
    dataloader: A Dataloader instance that needs to be trained on.
    loss_fn: A PyTorch Loss function for calculating the loss durning the training.
    optimizer: A Pytorch Optimizer to help reduce the loss function.
    device: A target device either 'cuda' or 'cpu'.
  Returns:
    A tuple of training loss and training accuracy metrics. 
  """
  # model in train mode
  model.train()
  # setup train_loss and train_acc
  train_loss, train_acc = 0, 0
  # Loop through dataloader
  for batch, (X, y) in enumerate(dataloader):
    # define the target device to data
    X, y = X.to(device), y.to(device)
    # forward pass
    y_pred = model(X)
    # loss and accuracy
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    # backward propagation    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # calculate accuracy metrics
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
  # Calculate the loss and accuracy for the model
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc

# function for testing the model
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """
  Test a PyTorch model.
  Turns the model to a evaluation mode and applies steps
  like loss and accuracy metrics.
  Args:
    model: A PyTorch model to be trained.
    dataloader: A Dataloader instance that needs to be trained on.
    loss_fn: A PyTorch Loss function for calculating the loss durning the training.
    device: A target device either 'cuda' or 'cpu'.
  Returns:
    A tuple of testing loss and testing accuracy metrics.  
  """
  model.eval()
  # setup train_loss and train_acc
  test_loss, test_acc = 0, 0
  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through dataloader
    for batch, (X, y) in enumerate(dataloader):
      # define the target device to data
      X, y = X.to(device), y.to(device)
      # forward pass
      test_pred_logits = model(X)
      # calculate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
      # calculate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
      # Calculate the loss and accuracy for the model
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

# Function for tracking different experiment
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
  """
  Creates a log directory using tensorboard, in a format runs/timestamp/experiment_name/model_name/extra.
  Args: 
    experiment_name: Name of experiment,
    model_name: Name of the model,
    extra: Anything extra to add to the directory. Default to None.
  Returns:
    torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir
  """
  timestamp = datetime.now().strftime("%d-%m-%Y") # time in format DD-MM-YYYY
  if extra:
    log_dir = os.path.join('runs', timestamp, experiment_name, model_name, extra)
  else:
    log_dir = os.path.join('runs', timestamp, experiment_name, model_name)
  print(f'\n[INFO] Creating SummaryWriter, saving to: {log_dir}...')
  return SummaryWriter(log_dir=log_dir)

# function for training and testing the model for n epochs.
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
  """
  Train and test a PyTorch Model

  Passing a model through train_step and test_step for certain
  number of epochs and training the model for that epoch loop.
  Calculate, print and stores all the metrics durning the training loop.

  Args:
    model: A pytorch model to be trained.
    train_dataloader: A dataloader only for train_step.
    test_dataloader: A dataloader only for test_step.
    loss_fn: A PyTorch Loss function for calculating the loss durning the training and testing.
    optimizer: A Pytorch Optimizer to help reduce the loss function while training the model.
    epochs: An integer indicating how much epochs to train the model for.
    device: A target device either 'cuda' or 'cpu'.
    writer: A SummaryWriter to save all the experiments.  

  Returns:
    A Dict containing all the values returned by train_step and test_step.
    results = {'train_loss': [],
               'train_acc': [],
               'test_loss': [],
               'test_acc': []}
  """
  # Creating a result dict.
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []}

  # Training and evaluation loop
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)
    print(f'Epoch: {epoch+1} | train_loss: {train_loss:.4f} , train_acc: {train_acc:.4f} | test_loss: {test_loss:.4f}, test_acc: {test_acc:.4f}')

    # store every epoch results in the results Dict
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

    # Use the writer to track experiment
    if writer:
      writer.add_scalars(main_tag='loss',
                         tag_scalar_dict={'train_loss': train_loss,
                                          'test_loss': test_loss},
                         global_step=epoch)
      writer.add_scalars(main_tag='Accuracy',
                         tag_scalar_dict={'train_acc': train_acc,
                                          'test_acc': test_acc},
                         global_step=epoch)
      # close the writer
      writer.close()
    else:
      pass

  return results

Writing engine.py


In [ ]:
import engine
from torch import nn 
from timeit import default_timer as timer

NUM_EPOCHS = 5

# creating loss, accuracy and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=baseline_model.parameters(), lr=0.001)
writer = engine.create_writer('trial', 'model0')

# Train model_0
model0_start_time = timer()
model_0_results = engine.train(baseline_model,
                               train_dataloaders,
                               test_dataloaders,
                               loss_fn,
                               optimizer,
                               NUM_EPOCHS,
                               device,
                               writer)
model_0_end_time = timer()
print(f'\nTotal Training Time: {model_0_end_time - model0_start_time:.3f} seconds')

NumExpr defaulting to 2 threads.

[INFO] Creating SummaryWriter, saving to: runs/02-12-2022/trial/model0...


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: 1 | train_loss: 1.0014 , train_acc: 0.4913 | test_loss: 1.0222, test_acc: 0.5000


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


...

Epoch: 3 | train_loss: 0.8843 , train_acc: 0.6034 | test_loss: 0.9210, test_acc: 0.5635


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: 4 | train_loss: 0.8459 , train_acc: 0.6373 | test_loss: 0.8804, test_acc: 0.6189


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: 5 | train_loss: 0.8081 , train_acc: 0.6595 | test_loss: 0.7865, test_acc: 0.6578

Total Training Time: 157.612 seconds


# 4. Save the model script - utils.py

In [ ]:
%%writefile utils.py
"""
Contains various utilities function for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """
  Saves a PyTorch Model to a directory.
  Args:
    model: A PyTorch model of nn.Module type.
    target_dir: A string of directory path.
    model_name: A filename for the saved model.
                Should include .pth or .pt as a file extention.
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith('.pth') or model_name.endswith('.pt'), 'model_name should end with .pth or .pt'
  model_save_path = target_dir_path / model_name

  # save the model state_dict
  print(f'\n[INFO] Saving Model to: {model_save_path}')
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing utils.py


In [ ]:
import utils
utils.save_model(baseline_model,
                 'models',
                 'baseline_model.pth')


[INFO] Saving Model to: models/baseline_model.pth


# 5. Train, evaluate and save the model - train.py

In [ ]:
%%writefile train.py
"""
Trains, evaluate and saves a PyTorch image classification model. 
"""

import os
import argparse
import torch
from torch import nn
import data_setup, engine, model_builder, utils
from torchvision import transforms
from datetime import datetime

# create a parser
parser = argparse.ArgumentParser(description='Get some hyperparameters')

# get experiment name
parser.add_argument('--exp_name',
                    default='experiment',
                    type=str,
                    help='Name of the experiment')
# get model name
parser.add_argument('--model_name',
                    default='model',
                    type=str,
                    help='Name of the model')
# get an arg for num_epochs
parser.add_argument("--num_epochs",
                    default=10,
                    type=int,
                    help='The number of epochs to train for')
# get an arg for batch_size
parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help='The number of sample for batch_size')
# get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help='The number of units for hidden layers')
# get an arg for learning_rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help='learning rate for optimizer')
# get an arg for training directory
parser.add_argument("--train_dir",
                    default='data/lion_tiger_wolf/train',
                    type=str,
                    help='The path for training data')
# get an arg for testing directory
parser.add_argument("--test_dir",
                    default='data/lion_tiger_wolf/val',
                    type=str,
                    help='The path for test data')
# get our arg from the parser
args=parser.parse_args()

# setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE=args.batch_size
NUM_WORKERS = os.cpu_count()
HIDDEN_UNITS=args.hidden_units
LEARNING_RATE= args.learning_rate
print(f'\n[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units with a learning rate of {LEARNING_RATE}')

# setup directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f'[INFO] Training data directory: {train_dir}')
print(f'[INFO] Testing data directory: {test_dir}')

# setup target device
device='cuda' if torch.cuda.is_available() else 'cpu'

# Create transform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create dataloaders using data_setup script
train_dataloaders, test_dataloaders, class_names =  data_setup.create_dataloaders(train_dir=train_dir,
                                                                                  test_dir=test_dir,
                                                                                  transform=data_transform,
                                                                                  batch_size=BATCH_SIZE,
                                                                                  num_workers=NUM_WORKERS)

# Create model using model_builder script
model = model_builder.TinyVGG(input_shape=3,
                              output_shape=len(class_names),
                              hidden_units=HIDDEN_UNITS).to(device)

# set loss, accuracy and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# define the summary writer and track our result through tensorboard
writer = engine.create_writer(args.exp_name,
                              args.model_name)

# start training using engine script
print(f'\n[INFO] Starting Model Training...')
engine.train(model=model,
             train_dataloader=train_dataloaders,
             test_dataloader=test_dataloaders,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device,
             writer=writer)

# save the model using utils script
utils.save_model(model=model,
                 target_dir='models',
                 model_name=args.model_name + '.pth')

Writing train.py


In [ ]:
!python train.py --num_epochs 5 --batch_size 10 --hidden_units 128 --learning_rate 0.0003


[INFO] Training a model for 5 epochs with batch size 10 using 128 hidden units with a learning rate of 0.0003
[INFO] Training data directory: data/lion_tiger_wolf/train
[INFO] Testing data directory: data/lion_tiger_wolf/val

[INFO] Creating SummaryWriter, saving to: runs/02-12-2022/experiment/model...

[INFO] Starting Model Training...
  0% 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
Epoch: 1 | train_loss: 0.9884 , train_acc: 0.5105 | test_loss: 0.9205, test_acc: 0.5505
 20% 1/5 [04:48<19:14, 288.52s/it]
Traceback (most recent call last):
  File "train.py", line 104, in <module>
    engine.train(model=model,
  File "/content/engine.py", line 156, in train
    train_loss, train_acc = train_step(model=model,
  File "/content/engine.py", line 45, in train_step
    loss.backward()
  File "/usr/local/lib/python3.8/dist-packages/torch/_tensor.py",

# 6. Creating a predict script - predict.py

In [ ]:
%%writefile predict.py
"""
Contains code to predict a single image class using a saved model.
"""
import os
import torch
import torchvision
import argparse
import model_builder

# creating a parser
parser = argparse.ArgumentParser()

# get an image_path
parser.add_argument('--image_path',
                    help='target image filepath to predict on')
# get a saved model
parser.add_argument("--model_path",
                    default='models/model.pth',
                    type=str,
                    help='target model to use for prediction filepath')
args = parser.parse_args()

# setup classs names
class_names = sorted([i for i in os.listdir('data/lion_tiger_wolf/train')])

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# get image path
IMG_PATH = args.image_path
print(f'[INFO] Predicting on image: {IMG_PATH}')

# Function to load the model
def load_model(filepath=args.model_path):
  # Need to use hyperparameters for saved model
  load_model = model_builder.TinyVGG(input_shape=3,
                                     output_shape=len(class_names),
                                     hidden_units=128).to(device)
  print(f'[INFO] Loading in model from: {filepath}')
  load_model.load_state_dict(torch.load(filepath))
  return load_model

def predict_on_image(image_path=IMG_PATH,
                     filepath=args.model_path):
  model=load_model(filepath)
  image = torchvision.io.read_image(str(image_path)).type(torch.float32)
  image = image/255.
  transform = torchvision.transforms.Resize(size=(64, 64))
  image = transform(image)

  model.eval()
  with torch.inference_mode():
    image=image.to(device)
    pred = model(image.unsqueeze(dim=0))
    pred_probs = torch.softmax(pred, dim=1)
    pred_label = torch.argmax(pred_probs, dim=1)
    pred_label_class = class_names[pred_label]
  print(f'[INFO] Pred class: {pred_label_class} and Pred Prob: {pred_probs.max():.3f}')

if __name__ == '__main__':
  predict_on_image()

Writing predict.py


In [ ]:
!python predict.py --image_path /content/data/lion_tiger_wolf/val/lion/05d42c9bd8.jpg

[INFO] Predicting on image: /content/data/lion_tiger_wolf/val/lion/05d42c9bd8.jpg
[INFO] Loading in model from: models/model.pth
Traceback (most recent call last):
  File "predict.py", line 61, in <module>
    predict_on_image()
  File "predict.py", line 45, in predict_on_image
    model=load_model(filepath)
  File "predict.py", line 40, in load_model
    load_model.load_state_dict(torch.load(filepath))
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 699, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'models/model.pth'


# 7. Creating a Image predict and plot script - image_plot.py

In [ ]:
%%writefile image_plot.py
"""
Contins code to predict and plot an image
"""
from typing import List, Tuple
from PIL import Image
import torch
import torchvision
import matplotlib.pyplot as plt

# defining device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# creating a function for predicting and plotting
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str],
                        image_size: Tuple[int, int],
                        transform: torchvision.transforms = None,
                        device: torch.device = device):
  """
  Predict and plot an image
  Args:
    model: A model to perform prediction.
    image_path: A path string of the image location.
    class_names: A list of all the classes names.
    image_size: A tuple with image size in shape of (height, width).
    transform: Torchvision transforms compose class to transform the data,
    device: A device either 'cuda' or 'cpu'.
  """
  # open image
  img = Image.open(image_path)
  # create transformation
  if transform is not None:
    img_transform = transform
  else:
    img_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(image_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
    ])
  # predict image
  model.to(device)
  model.eval()
  with torch.inference_mode():
    img_transform = img_transform(img).unsqueeze(dim=0)
    img_pred = model(img_transform.to(device))
  img_pred_probs = torch.softmax(img_pred, dim=1)
  img_pred_label = torch.argmax(img_pred_probs, dim=1)
  # plot image
  plt.figure()
  plt.imshow(img)
  plt.title(f'Pred: {class_names[img_pred_label]} | Prob: {img_pred_probs.max():.3f}')
  plt.axis(False);

Overwriting image_plot.py


# Copying the python file to drive

In [ ]:
import shutil
import os
for i in os.listdir('/content/'):
  if i.endswith('.py'):
    shutil.copy2(i, '/content/drive/MyDrive/Colab Notebooks/My Project/Image Classification/Animal Vision/model_scripts')